# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.  

Известно:  

сalls — количество звонков  
minutes — суммарная длительность звонков в минутах  
messages — количество sms-сообщений  
mb_used — израсходованный интернет-трафик в Мб  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)  

## Оглавление  

[1.Общая информация о содержимых в файле данных](#шаг_1)  
[2.Делим данные на выборки  ](#шаг_2)  
[3.Исследуем модели  ](#шаг_3)  
[4.Проверяем модель на тестовой выборке](#шаг_4)  
[5.Проверяем можель на адекватность](#шаг_5)  


## 1. Откройте и изучите файл <a id="шаг_1"></a>

In [1]:
#тут все нужные библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#открываем файл
data = pd.read_csv('/datasets/users_behavior.csv')


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.sample(5)

,calls,minutes,messages,mb_used,is_ultra
2555,31.0,252.77,21.0,13556.92,0
1347,23.0,147.69,30.0,18417.93,0
483,77.0,593.45,0.0,20770.04,0
2123,34.0,252.87,26.0,24550.06,1
2725,44.0,342.11,25.0,18110.91,0


In [5]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

По условию:«Ультра» — 1, «Смарт» — 0, т.е. перекос в пользу смарта более чем в два раза.  В остальном с данными все корректно -пропусков и дубликатов нет, предобработка не требуется.  
Целевой признак – is_ultra, т.е. модель должна предсказывать значение 1 если клиенту нужно предложить тариф "Ультра" или 0 если предлагается тариф "Смарт". Целевой признак является категориальным.

## 2. Разбейте данные на выборки <a id="шаг_2"></a>

Разобьем данные на три части: обучающую, валидационную и тестовую. Т.к. тестовой выборки изначально нет, то делим в пропорции  60%-20%-20%.

In [6]:
# обучающая выборка = 60% 
data_train, data_valid = train_test_split(data, test_size=0.4, random_state=12345)
# валидационная и тестовая по 20% каждая
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=12345)

In [7]:
data_train.shape[0]


1928

In [8]:
data_valid.shape[0]

643

In [9]:
data_test.shape[0]

643

In [10]:
#проверим
data_train.count() + data_valid.count() + data_test.count()

calls       3214
minutes     3214
messages    3214
mb_used     3214
is_ultra    3214
dtype: int64

Все разбили корректно.

## 3. Исследуйте модели <a id="шаг_3"></a>

#### Для решения задачи классификации рассмотрим изученные модели:

Дерево решений (DecisionTreeClassifier)  
Случайный лес (RandomForestClassifier)  
Логистическую регрессию (LogisticRegression)  

In [11]:
#Присвоим переменным признаки и целевые признаки
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

#Валидационная выборка
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

#Обучающая выборка
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

#Тестовая выборка
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

#### Дерево решений (DecisionTreeClassifier)

In [12]:
for depth in range(1, 11):
    model_DecisionTreeClassifier = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_DecisionTreeClassifier.fit(features_train, target_train)
    predictions = model_DecisionTreeClassifier.predict(features_valid)
    accuracy_DecisionTreeClassifier = accuracy_score(target_valid, predictions)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_DecisionTreeClassifier)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633


Самая высокая оценка правильности 0.7853810264385692, когда задан гиперпараметр глубины равный 3.

#### Случайный лес (RandomForestClassifier)

In [13]:
for estim in range(10, 100, 10):
    model_RandomForestClassifier = RandomForestClassifier(random_state=12345, n_estimators=estim)
    model_RandomForestClassifier.fit(features_train, target_train)
    predictions = model_RandomForestClassifier.predict(features_valid)
    accuracy_RandomForestClassifier = accuracy_score(target_valid, predictions)
    print("estim =", estim, accuracy_RandomForestClassifier)

estim = 10 0.7853810264385692
estim = 20 0.7869362363919129
estim = 30 0.7838258164852255
estim = 40 0.7838258164852255
estim = 50 0.7916018662519441
estim = 60 0.7853810264385692
estim = 70 0.7791601866251944
estim = 80 0.7822706065318819
estim = 90 0.7838258164852255


И результат 0.7916018662519441 при 50 деревьях.

#### Логистическая регрессия (LogisticRegression)

In [14]:
model_LogisticRegression = LogisticRegression(random_state=12345)
model_LogisticRegression.fit(features_train, target_train)
predictions = model_LogisticRegression.predict(features_valid)
accuracy_LogisticRegression = accuracy_score(target_valid, predictions)
accuracy_LogisticRegression

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7589424572317263

Самое низкое значение  0.7589424572317263

#### Сравним все модели

И так, соберем все лучшие значения:  
Дерево решений (DecisionTreeClassifier): 0.7853810264385692  
Случайный лес (RandomForestClassifier): 0.7916018662519441  
Логистическая регрессия (LogisticRegression): 0.7589424572317263

## 4. Проверьте модель на тестовой выборке <a id="шаг_4"></a>

In [15]:
model = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=12345)
model.fit(features_train, target_train)
predictions= model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print('Качество модели на тестовой выборке:',accuracy)

Качество модели на тестовой выборке: 0.7776049766718507


## 5. (бонус) Проверьте модели на адекватность <a id="шаг_5"></a>

Посчитаем количество перешедших на тариф "Cмарт" к общему количеству.

In [16]:
data[data.is_ultra == 0].count() / data.count()

calls       0.693528
minutes     0.693528
messages    0.693528
mb_used     0.693528
is_ultra    0.693528
dtype: float64

In [17]:
features_data = data.drop(['is_ultra'], axis = 1)
target_data = data['is_ultra']
model_RandomForestClassifier = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=12345)
model_RandomForestClassifier.fit(features_data, target_data)
predictions_data = model_RandomForestClassifier.predict(features_data)
accuracy_data = accuracy_score(target_data, predictions_data)
print('Качество модели при проверке на адекватность:',accuracy_data)


Качество модели при проверке на адекватность: 0.8018046048537648


#### Вывод:  
Проверка на адекватность показала,что результат работы обученной модели лучше случайных результатов. accuracy не меньше 0.75

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
